In [131]:
import pandas as pd
#pd.set_option("display.max_rows",200)             # to display ass row
#pd.set_option("display.max_rows",None)             # to display ass columns

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import collections 

In [132]:
movies_df = pd.read_csv(r"C:\Users\91752\OneDrive - DIT University\Desktop\ML projects\movie_recomended\tmdb_5000_movies.csv")
credits_df = pd.read_csv(r"C:\Users\91752\OneDrive - DIT University\Desktop\ML projects\movie_recomended\tmdb_5000_credits.csv")

df = movies_df.merge(credits_df,on='title')

###    preprocessing    ###

In [133]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [134]:
df = df[["movie_id","title","overview","genres","keywords","cast","crew"]]

In [135]:
df.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [136]:
df.dropna(inplace=True)

In [137]:
df.duplicated().sum()

0

In [138]:
df.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [139]:
import ast 
ast.literal_eval(df.iloc[0].genres)

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [140]:
def convert(obj):
    li=[]
    for i in ast.literal_eval(obj):
        li.append(i['name'])
    return li

In [141]:
df['genres'] = df['genres'].apply(convert)

In [142]:
df['keywords'] = df['keywords'].apply(convert)

In [143]:
def convert2(obj):
    li=[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter !=3:
            li.append(i['name'])
            counter+=1
        else:
            break
    return li

In [144]:
df['cast'] = df['cast'].apply(convert2)

In [145]:
def fetch_director(obj):
    li=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            li.append(i['name'])
            break
    return li

In [146]:
df['crew'] = df['crew'].apply(fetch_director)

In [147]:
df['overview'] = df['overview'].apply(lambda x:x.split())

In [148]:
df['genres'] = df['genres'].apply(lambda x : [i.replace(" ","") for i in x])
df['keywords'] = df['keywords'].apply(lambda x : [i.replace(" ","") for i in x])
df['cast'] = df['cast'].apply(lambda x :[i.replace(" ","") for i in x])
df['crew'] = df['crew'].apply(lambda x : [i.replace("","") for i in x])

In [149]:
df['tags'] = df['overview'] + df['genres'] + df['keywords'] + df['cast'] + df['crew'] 

In [150]:
df = df[['movie_id' , 'title' , 'tags']]

In [151]:
df['tags'] = df['tags'].apply(lambda x : " ".join(x))

In [153]:
df['tags'] = df['tags'].apply(lambda x:x.lower())

###    vectorization    ###

In [154]:
from sklearn.feature_extraction.text import CountVectorizer

In [174]:
cv = CountVectorizer(max_features=5000,stop_words="english")
vectors = cv.fit_transform(df['tags']).toarray()

In [179]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zooeydeschanel', 'zucker', 'zwick'],
      dtype=object)

In [177]:
#demo = pd.DataFrame(vectors,columns=cv.get_feature_names_out())
#demo.sum().sort_values(ascending=False)

In [171]:
import nltk                    # NLP  # stemming
from nltk.stem.porter import PorterStemmer

In [172]:
ps = PorterStemmer()

def stem(text):
    li=[]
    for i in text.split():
        li.append(ps.stem(i))
    return " ".join(li)


In [173]:
df['tags'] = df['tags'].apply(stem)

In [180]:
from sklearn.metrics.pairwise import cosine_similarity

In [186]:
similarity = cosine_similarity(vectors)

In [ ]:
#sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6] 

In [196]:
def recommend (movie):
    movie_index = df[df['title'] == movie].index[0]     # getting movie index number
    distances = similarity[movie_index]                 # finding distances of all movies from our movie
    movie_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]        # sorting based on index number 
    for i in movie_list:
        print(df.iloc[i[0]].title)

In [198]:
recommend('Meet Dave') 

The Hitchhiker's Guide to the Galaxy
Escape from Planet Earth
Planet 51
Home
Aliens in the Attic
